# User-based Collaborative Filtering

User-based k-Nearest Neighbors (UserKNN) is another collaborative filtering algorithm used in
recommendation systems. The core idea behind UserKNN is to make recommendations based on the
preferences of similar users. UserKNN is intuitive and leverages the idea that users who have rated
items similarly in the past will continue to have similar preferences. It’s especially effective when users
have a rich history of interactions.

In [38]:
# %pip install import_ipynb
import import_ipynb
import cornac
from cornac.data import Dataset
import cornac.metrics as met
from cornac.eval_methods import BaseMethod
from data_loader import DataLoader # type: ignore

In [33]:
data_path = "data/"
data_loader = DataLoader(data_path)

In [34]:
train_dataset, test_dataset = data_loader.load_for_cornac(dataset_type='split')
print(train_dataset.shape)
train_dataset.head()

(513384, 4)


,ReviewId,RecipeId,AuthorId,Rating
0,826743,3745,345380,4
1,1247176,26217,406131,1
2,1250914,17123,355582,5
3,183560,123283,58104,4
4,1255493,110139,383795,5


## User-based Collaborative Filtering (UserKNN)

User-based collaborative filtering finds users who are similar to the target user and recommends items that these similar users have liked. The algorithm works as follows:

1. **Find Similar Users**: Calculate similarity between users based on their rating patterns
2. **Neighborhood Selection**: Select k most similar users (k-nearest neighbors)
3. **Prediction**: Predict ratings for unrated items based on similar users' ratings
4. **Recommendation**: Recommend items with highest predicted ratings

### Key Parameters:
- **k**: Number of similar users to consider (neighborhood size)
- **similarity**: Similarity metric (cosine, pearson, etc.)
- **min_support**: Minimum number of common items between users

In [41]:
metrics = [met.RMSE(), met.Precision(k=10), met.Recall(k=10)]
eval_method = BaseMethod.from_splits(train_dataset[['AuthorId', 'RecipeId', 'Rating']].values, test_dataset[['AuthorId', 'RecipeId', 'Rating']].values)

In [ ]:
from cornac.models import UserKNN
from cornac.eval_methods import RatioSplit
import pandas as pd
import time

cornac_train_dataset = Dataset.from_uir(train_dataset[['AuthorId', 'RecipeId', 'Rating']].values.tolist(), seed=42)
cornac_test_dataset = Dataset.from_uir(test_dataset[['AuthorId', 'RecipeId', 'Rating']].values.tolist(), seed=42)

k_values = [10, 20, 50, 100]
user_knn_results = []

print("\n" + "="*80)
print("USER-BASED COLLABORATIVE FILTERING (UserKNN) EVALUATION")
print("="*80)

for k in k_values:
    print(f"\nTesting UserKNN with k={k}...")
    user_knn = UserKNN(k=k, similarity='cosine', verbose=True)
    start_time = time.time()
    user_knn.fit(cornac_train_dataset)
    total_time = time.time() - start_time
    # Evaluate metrics
    results = eval_method.evaluate(user_knn, metrics=metrics, user_based=False)
    for metric in results:
        print(f"  {metric.name}: {metric.value:.4f}")
    """model_result = {
        'model': f'UserKNN(k={k})',
        'k': k,
        'rmse': rmse,
        'precision': precision,
        'recall': recall,
        'total_time': total_time
    }
    user_knn_results.append(model_result)
    print(f"  RMSE: {rmse:.4f}")
    print(f"  Precision@10: {precision:.4f}")
    print(f"  Recall@10: {recall:.4f}")
    print(f"  Total time: {total_time:.2f}s")"""

# Display results in a nice table
"""user_knn_df = pd.DataFrame(user_knn_results)
print(f"\n{'='*80}")
print("USER-BASED COLLABORATIVE FILTERING RESULTS SUMMARY")
print(f"{'='*80}")
print(user_knn_df.to_string(index=False, float_format='%.4f'))"""


USER-BASED COLLABORATIVE FILTERING (UserKNN) EVALUATION

Testing UserKNN with k=10...


100%|██████████| 17748/17748 [00:01<00:00, 10084.69it/s]
/home/r-one/.local/lib/python3.11/site-packages/cornac/models/recommender.py:322: UserWarning: Model is already fitted. Re-fitting will overwrite the previous model.
  warnings.warn(
100%|██████████| 17748/17748 [00:00<00:00, 19445.86it/s]


AttributeError: 'Result' object has no attribute 'value'